In [6]:
!pip install ray[tune] tune-sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 700.9 kB/s eta 0:00:00a 0:00:01


In [9]:
import torch

data, y = torch.load('data/data_v2.pkl')

In [10]:
train_data, test_data = data[:len(y)], data[len(y):]


In [11]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )

In [13]:
X_train

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q,Embarked_S,Title_Capt.,...,Title_Major.,Title_Master.,Title_Miss.,Title_Mlle.,Title_Mme.,Title_Mr.,Title_Mrs.,Title_Ms.,Title_Rev.,Title_Sir.
PassengerId,,,,,,,,,,,,,,,,,,,,,
332,1,45.500000,0,0,3.349904,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
734,2,23.000000,0,0,2.564949,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
383,3,32.000000,0,0,2.070022,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
705,3,26.000000,1,0,2.061048,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
814,3,6.000000,4,2,3.442819,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,3,21.000000,0,0,2.034706,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
271,1,32.252151,0,0,3.433987,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
861,3,41.000000,2,0,2.646763,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# {'svc__C': 0.34263484415417084, 'svc__gamma': 0.22319380707375788}
std = StandardScaler()
svc = SVC()
pipe = Pipeline(
    [('std', std),
     ('svc', svc)]
)
param = {'svc__C': 1.47, 'svc__gamma': 0.15923601840597726}
# pipe.set_params(**param)
pipe.fit(X_train, y_train)
print(pipe.score(X_train, y_train), pipe.score(X_test, y_test))
cross_val_score(pipe,X_train,y_train,cv=10,n_jobs=-1,verbose=2).mean()


0.8441011235955056 0.8044692737430168


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


0.8328834115805946

In [43]:
param = {'svc__C': [1.5], 'svc__gamma': np.arange(0, 5, 0.01)}
grid_search = GridSearchCV(pipe, param_grid=param, verbose=2, n_jobs=-1, cv=10)
grid_search.fit(train_data, y)
grid_search.best_score_, grid_search.best_params_

Fitting 10 folds for each of 500 candidates, totalling 5000 fits


(0.8327715355805243, {'svc__C': 1.5, 'svc__gamma': 0.16})

In [11]:
pipe.fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.8441011235955056, 0.8268156424581006)

In [12]:
pipe.fit(train_data, y)
pipe.score(train_data, y)

0.8439955106621774

In [10]:
# 超参数搜索
from ray import tune
from ray.tune.sklearn import TuneSearchCV

param = {'svc__C': tune.loguniform(1e-2, 100),
         'svc__gamma': tune.uniform(1e-6, 10)}

tune_search = TuneSearchCV(
    pipe,
    param,
    search_optimization="bayesian",
    n_trials=-1,
    verbose=1,
    return_train_score=True,
    mode='max',
    time_budget_s=600,
    error_score=np.nan,
    use_gpu=True,
    cv=10
)
tune_search.fit(train_data, y)
print(tune_search.best_params_)

2022-05-12 13:00:59,022	INFO tune.py:702 -- Total run time: 602.81 seconds (602.30 seconds for the tuning loop).


{'svc__C': 0.34263484415417084, 'svc__gamma': 0.22319380707375788}


In [45]:
import pandas as pd

# best_model = tune_search.best_estimator
best_model = grid_search.best_estimator_
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('svc.csv')
# torch.save(tune_search.best_params_, 'svc.pkl')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
